In [ ]:
path_name = 'data'

In [ ]:
upper_path = 'data-testing'

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [ ]:
#Fetch 1 car image for display
car1 = cv2.imread('../input/'+upper_path+'/'+path_name+'/'+path_name+'/training/00-damage/0050.JPEG')

In [ ]:
import cv2
import numpy as np
from PIL import Image
def myFunc(image):
    hsv_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    return hsv_image

In [ ]:
#Keras image generator will be used to create additional training images

image_gen = ImageDataGenerator(rotation_range=35, # rotate the image 30 degrees
                               width_shift_range=0.05, # Shift the pic width by a max of 10%
                               height_shift_range=0.05, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.05, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.1, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest'
                          #    ,
                          #     n = myFunc # Fill in missing pixels with the nearest filled value
                              # featurewise_center=True
                              )

In [ ]:

image_gen_validate = ImageDataGenerator(
                               rescale=1/255
    #,
     #                          preprocessing_function = myFunc # Rescale the image by normalzing it.       
                               #featurewise_center=True
                              )

In [ ]:
#image_gen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

In [ ]:
#image_gen_validate.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

In [ ]:
#Show an image of one of the damaged cars
plt.imshow(car1)

In [ ]:
#Example Transformation of car image.
plt.imshow(image_gen.random_transform(car1))

In [ ]:
image_gen.flow_from_directory('../input/'+upper_path+'/'+path_name+'/'+path_name+'/training')

In [ ]:
image_gen.flow_from_directory('../input/'+upper_path+'/'+path_name+'/'+path_name+'/validation')

In [ ]:
#Resizing images to 224x224.  I'm thinking of using VGG-16 for transfer learning so trying out this.  Average image size is 312 x 447
# width,height,channels
image_shape = (299,299,3)
batch_size = 10
#batch_size = 5

In [ ]:
from keras import optimizers
from keras import backend as K

In [ ]:
import numpy as np 
from keras.models import Model 
from keras.applications import Xception 
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D 
#from keras.optimizers import Adam
from keras.applications import imagenet_utils 
from keras.utils import np_utils 
from keras.callbacks import EarlyStopping 

In [ ]:
inception_model = InceptionV3(weights='imagenet', include_top=False) 

In [ ]:
from keras.layers import BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers import Dropout

x = inception_model.output 
x = GlobalAveragePooling2D()(x)
#x = GlobalMaxPooling2D()(x)
x = Dense(768, activation='relu')(x) 
x = BatchNormalization()(x)
out = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=inception_model.input, outputs=out)

In [ ]:
for layer in inception_model.layers:
   layer.trainable = False

In [ ]:
for i, layer in enumerate(model.layers):
  print(i, layer.name) 

In [ ]:
for layer in model.layers[:156]:
    layer.trainable = False
for layer in model.layers[156:]:
    layer.trainable = True

In [ ]:
len(model.layers)

In [ ]:
print("Trainable layers:", model.trainable_weights)

# Create the CNN

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer

model.compile(loss='binary_crossentropy',
              #optimizer=optimizers.RMSprop(lr=0.0001),
              #optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
              #optimizer='Adagrad',
              optimizer=optimizers.adam(lr=0.0001),
              #optimizer=optimizers.nadam(lr=0.0001),
              metrics=['accuracy'])

model.summary()


In [ ]:
validation_size = image_gen.flow_from_directory('../input/'+upper_path+'/'+path_name+'/'+path_name+'/validation').n
training_size = image_gen.flow_from_directory('../input/'+upper_path+'/'+path_name+'/'+path_name+'/training').n

In [ ]:
#Train the model!
train_image_gen = image_gen.flow_from_directory('../input/'+upper_path+'/'+path_name+'/'+path_name+'/training',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

train_image_gen.class_indices

test_image_gen = image_gen_validate.flow_from_directory('../input/'+upper_path+'/'+path_name+'/'+path_name+'/validation',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

train_image_gen.class_indices

epochs = 150

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='acc', mode='min', verbose=1, patience=6)

#adding a LR reduction callback.  This will reduce the learning rate if the validation loss is not improving after 5 epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.00008)
from keras.callbacks import ModelCheckpoint
mc =  ModelCheckpoint('Inception.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=10)

#Run the model!
results = model.fit_generator(train_image_gen,epochs=epochs,
                              steps_per_epoch=training_size/batch_size,
                              validation_data=test_image_gen,
                             validation_steps=validation_size/batch_size,
                             callbacks=[mc])

<a href="Inception.h5"> Download File </a>

In [ ]:
import pandas as pd
my_final_results = pd.DataFrame(results.history['acc'])

In [ ]:
import os
dir = './'
content = os.listdir(dir)
print(content)

In [ ]:
my_final_results.to_csv('results.csv')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Transfer Learning Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,epochs+1))
ax1.plot(epoch_list, results.history['acc'], label='Train Accuracy')
ax1.set_ylim(ymin=0)
ax1.plot(epoch_list, results.history['val_acc'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, epochs+1, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, results.history['loss'], label='Train Loss')
ax2.plot(epoch_list, results.history['val_loss'], label='Validation Loss')
ax2.set_ylim(ymin=0)
ax2.set_xticks(np.arange(0, epochs+1, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
# save model
model.save('final_model_x.h5')


In [ ]:
# load the saved model
from keras.models import load_model

model = load_model('best_model_transfer_x.h5')